In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import transformers
import torch
import matplotlib.pyplot as plt

import sys

LIBRARY_PATH = "/p/project/deepacf/maelstrom/ehlert1/a2/src/"
sys.path.append(LIBRARY_PATH)
import a2.training.training_hugging
import a2.training.evaluate_hugging
import a2.training.dataset_hugging
import a2.plotting.analysis
import a2.plotting.histograms
import a2.dataset
import a2.utils

In [ ]:
a2.training.utils_training.gpu_available()

In [ ]:
BASE_FOLDER_MODELS = a2.utils.file_handling.get_folder_models()
FOLDER_MODEL_PRETRAINED = (
    BASE_FOLDER_MODELS / "/model_weights/output_rainprediction_simpledeberta/era5/checkpoint-7617/"
)
FOLDER_MODEL = BASE_FOLDER_MODELS / "llama-13b-hf"
FOLDER_TWEETS = a2.utils.file_handling.get_folder_data() / "tweets/"
FILE_TWEETS = FOLDER_TWEETS / "2017_2020_tweets_keywords_near_station.nc"
FOLDER_EMBEDDINGS = "/home/kristian/Projects/a2/data/embeddings/cls_token/"
FILE_EMBEDDINGS = FOLDER_EMBEDDINGS + "cls_tokenstweets_2017_era5_normed_filtered.nc.npy"
!ls $FILE_TWEETS

In [ ]:
ds = a2.dataset.load_dataset.load_tweets_dataset(FILE_TWEETS)
ds["raining"] = (["index"], np.array(ds.station_tp_mm.values > 0, dtype=int))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!cat $FOLDER_MODEL/tokenizer_config.json

In [ ]:
transformers.__version__

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(FOLDER_MODEL)

In [ ]:
model = transformers.LlamaForCausalLM.from_pretrained(FOLDER_MODEL)

In [ ]:
text = r'Assign a probability that it is raining to the following tweets. The content of the tweets should hint something about the weather being rainy and not good in general. Tweet 1: "Well that last rumble of thunder made the house shake, I wasn\'t scared for a couple of seconds" Tweet 2: "#viewfromthe office what a great morning @Grantham and District". Return the results in a json file like: [ { "tweet": 1, "content": "Well that last rumble of thunder made the house shake, I wasn\'t scared for a couple of seconds", "explanation": "short explanation of the rain probability based on content of this tweet", "rain_probability": x.x }, ... ] Result: [ { "tweet": 1, "content":'
text = r"""Assign a probability that it is raining to the following tweets. The content of the tweets should hint something about the weather being rainy and not good in general. 
Tweet 1: "The sound of rain tapping on the window" 
Tweet 2: "Boris seems desperate for the rain to finish". 
Return the results in a json file like: [ { "tweet": 1, "content": "The sound of rain tapping on the window", "explanation": "The sound of rain heard implies that is raining.", "rain_probability": 0.9 }, ... ] 
Result: [ { "tweet": 1, "content":"""
input_ids = tokenizer.encode(text, return_tensors="pt")

sample_outputs = model.generate(
    input_ids,
    temperature=0.2,
    do_sample=True,
    max_length=400,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
    early_stopping=True,
)

print("Output:\n" + 100 * "-")
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

In [ ]:
?model.prepare_inputs_for_generation